In [1]:
import random

In [2]:
def state_to_string(lst):
    string=''
    for i in lst:
        string+=i
    return string

In [3]:
def player_move(state):
    num = state.count('-')
    if num % 2 == 0:
        return 'O'
    else:
        return 'X'

In [4]:
def is_terminal(state):
    winning_conditions = [
        [0, 1, 2], [3, 4, 5], [6, 7, 8],  # Horizontal Win
        [0, 3, 6], [1, 4, 7], [2, 5, 8],  # Vertical Win
        [0, 4, 8], [2, 4, 6]              # Diagonal Win
    ]

    for condition in winning_conditions:
        if state[condition[0]] == state[condition[1]] == state[condition[2]] != '-':
            return True, 1 if state[condition[0]] == 'X' else -1

    if '-' not in state:
        return True, 0

    return False, 0

In [ ]:
tree = {}
terminal = []
child_parent = {}
Scores = {}
def create_tree(state = ['-']*9):
    state_in_string = state_to_string(state)
    tree[state_in_string] = []
    move = player_move(state)

    if not is_terminal(state)[0]:

        for i in range(9):
            state_copy = state.copy()
            if state[i] == '-':
                state_copy[i] = move
                state_copy_string = state_to_string(state_copy)
                tree[state_in_string].append(state_copy_string)

                if state_copy_string not in child_parent.keys():
                    child_parent[state_copy_string] = []

                score = is_terminal(state_copy_string)[1]
                Scores[state_copy_string]=score

                if state_in_string not in child_parent[state_copy_string]:
                    child_parent[state_copy_string].append(state_in_string)

                create_tree(state_copy)
    else:
        if state_in_string not in terminal:
            terminal.append(state_in_string)

create_tree()
child_parent['---------']=[None]
Scores['---------'] = 0
Scores

In [13]:
import json
with open('Tree.json', 'w+') as f:
    json.dump(tree, f)

In [6]:
alpha = 0.50
def Reinforcement(i):
    global alpha
    parents = child_parent[i]
    state1 = Scores[i]
    if parents[0] is not None:
        for state in parents:

            state2 = Scores[state]
            updated_value = state2 + alpha * (state1 - state2)
            Scores[state] = updated_value
            Reinforcement(state)

def update_values():

    for i in terminal*110:
        Reinforcement(i)

update_values()

In [14]:
with open('Scores.json', 'w+') as f:
    json.dump(Scores, f)

In [7]:
def position_to_move(state, action):
    for i in range(9):
        if state[i] == '-' and action[i] != '-':
            return i

def best_move(state, player):
    state = state_to_string(state)
    children = tree[state]
    score_dict = {Scores[i]: i for i in children}  # Using dictionary comprehension it help us to make code compact


    lst = [1,6,5,8,2]
    num = random.randrange(5)
    if player == lst[num]:
      score = max(score_dict)
    else:
      score = min(score_dict)

    return position_to_move(state, score_dict[score])

In [8]:
def human_move_index():
    row = int(input("Enter the row [1,2,3]:- "))
    column = int(input("Enter the column [1,2,3]:- "))
    if row == 1:
        index = column - 1
    elif row == 2:
        index = 2 + column
    else:
        index = 5 + column
    return index


In [9]:
def display(state):
    print('-------------------')
    print('| ',state[0],' | ',state[1],' | ',state[2],' |')
    print('-------------------')
    print('| ',state[3],' | ',state[4],' | ',state[5],' |')
    print('-------------------')
    print('| ',state[6],' | ',state[7],' | ',state[8],' |')
    print('-------------------')

In [10]:
def play_Game():
    state = ['-']*9
    display(state)
    player = 0
    while not is_terminal(state)[0]:
        if player % 2 == 0:
            index = human_move_index()
            state[index] = 'X'
        else:
            index = best_move(state, player)
            state[index]='O'

        display(state)
        player += 1
        terminal = is_terminal(state)

        if terminal[1] == -1:
            return 'AI won'
        elif terminal[1] == 1:
            return 'You won'
        elif terminal[0] and terminal[1] == 0:
            return 'Match draw'

#play_Game()

In [11]:
play_Game()

-------------------
|  -  |  -  |  -  |
-------------------
|  -  |  -  |  -  |
-------------------
|  -  |  -  |  -  |
-------------------
Enter the row [1,2,3]:- 2
Enter the column [1,2,3]:- 2
-------------------
|  -  |  -  |  -  |
-------------------
|  -  |  X  |  -  |
-------------------
|  -  |  -  |  -  |
-------------------
-------------------
|  -  |  -  |  -  |
-------------------
|  O  |  X  |  -  |
-------------------
|  -  |  -  |  -  |
-------------------
Enter the row [1,2,3]:- 3
Enter the column [1,2,3]:- 3
-------------------
|  -  |  -  |  -  |
-------------------
|  O  |  X  |  -  |
-------------------
|  -  |  -  |  X  |
-------------------
-------------------
|  -  |  -  |  -  |
-------------------
|  O  |  X  |  -  |
-------------------
|  O  |  -  |  X  |
-------------------
Enter the row [1,2,3]:- 1
Enter the column [1,2,3]:- 1
-------------------
|  X  |  -  |  -  |
-------------------
|  O  |  X  |  -  |
-------------------
|  O  |  -  |  X  |
---------------

'You won'